In [8]:
# Import necessary libraries
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.stats import norm

# Function to calculate the expected return and volatility
def calculate_portfolio_metrics(tickers, weights, start_date, end_date):
    # Download historical stock data from Yahoo Finance
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    
    # Calculate daily returns
    returns = data.pct_change().dropna()
    
    # Calculate expected annual return (assuming 252 trading days in a year)
    expected_annual_return = np.sum(np.mean(returns, axis=0) * weights) * 252
    
    # Calculate portfolio volatility (annualized)
    covariance_matrix = returns.cov() * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    
    return expected_annual_return, portfolio_volatility

# Function to calculate the Sharpe Ratio
def calculate_sharpe_ratio(expected_return, volatility, risk_free_rate=0.01):
    sharpe_ratio = (expected_return - risk_free_rate) / volatility
    return sharpe_ratio

# Function to calculate the Value at Risk (VaR)
def calculate_var(returns, weights, confidence_level=0.95):
    portfolio_returns = np.dot(returns, weights)
    mean_return = np.mean(portfolio_returns)
    portfolio_std_dev = np.std(portfolio_returns)
    
    # Use the inverse of the normal distribution to calculate VaR
    z_score = norm.ppf(1 - confidence_level)
    var = mean_return + z_score * portfolio_std_dev
    return var

# Function to calculate portfolio beta
def calculate_portfolio_beta(portfolio_tickers, portfolio_weights, benchmark_ticker, start_date, end_date):
    # Download historical stock and benchmark data
    portfolio_data = yf.download(portfolio_tickers, start=start_date, end=end_date)['Adj Close']
    benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)['Adj Close']
    
    # Convert index to timezone-naive for both portfolio and benchmark data
    portfolio_data.index = portfolio_data.index.tz_localize(None)
    benchmark_data.index = benchmark_data.index.tz_localize(None)
    
    # Calculate daily returns
    portfolio_returns = portfolio_data.pct_change().dropna()
    benchmark_returns = benchmark_data.pct_change().dropna()
    
    # Calculate portfolio returns
    portfolio_total_returns = portfolio_returns.dot(portfolio_weights)
    
    # Combine the portfolio returns with the benchmark
    combined_data = pd.concat([portfolio_total_returns, benchmark_returns], axis=1).dropna()
    combined_data.columns = ['Portfolio', 'Benchmark']
    
    # Calculate the covariance between the portfolio and the benchmark
    covariance_matrix = combined_data.cov()
    portfolio_beta = covariance_matrix.loc['Portfolio', 'Benchmark'] / covariance_matrix.loc['Benchmark', 'Benchmark']
    
    return portfolio_beta


# Main function to monitor the portfolio
def monitor_portfolio(tickers, weights, start_date, end_date, confidence_level=0.95, risk_free_rate=0.01):
    # Calculate portfolio metrics
    expected_return, portfolio_volatility = calculate_portfolio_metrics(tickers, weights, start_date, end_date)
    
    # Calculate Sharpe ratio
    sharpe_ratio = calculate_sharpe_ratio(expected_return, portfolio_volatility, risk_free_rate)
    
    # Calculate Value at Risk (VaR)
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    returns = data.pct_change().dropna()
    var = calculate_var(returns, weights, confidence_level)
    
    # Calculate portfolio beta against a benchmark (e.g., S&P 500)
    portfolio_beta = calculate_portfolio_beta(tickers, weights, 'DJIA', start_date, end_date)
    
    return {
        'Expected Return': expected_return,
        'Portfolio Volatility': portfolio_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Value at Risk (VaR)': var,
        'Portfolio Beta': portfolio_beta
    }


# Function to calculate the covariance matrix and diversification measures
def calculate_diversification_metrics(tickers, weights, start_date, end_date):
    # Download historical data for the tickers
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    
    # Calculate daily returns
    returns = data.pct_change().dropna()
    
    # Calculate the covariance matrix
    covariance_matrix = returns.cov() * 252  # Annualized covariance matrix (252 trading days)
    
    # Calculate the correlation matrix
    correlation_matrix = returns.corr()
    
    # Portfolio variance
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    
    portfolio_volatility = np.sqrt(portfolio_variance)
    
    # Individual asset volatilities (standard deviations)
    asset_volatilities = np.sqrt(np.diag(covariance_matrix))
    
    # Diversification ratio: sum of weighted individual volatilities / portfolio volatility
    diversification_ratio = np.sum(weights * asset_volatilities) / portfolio_volatility
    
    return {
        'Covariance Matrix': covariance_matrix,
        'Correlation Matrix': correlation_matrix,
        'Portfolio Variance': portfolio_variance,
        'Portfolio Volatility': portfolio_volatility,
        'Diversification Ratio': diversification_ratio
    }

# Example usage of the diversification metrics function
if __name__ == "__main__":
    tickers = ['NKE', 'amzn']  # Example tickers
    weights = np.array([1, 0])  # Equal weights for simplicity
    start_date = datetime.today() - timedelta(days=365)
    end_date = datetime.today()

    # Call the monitor_portfolio function
    portfolio_metrics = monitor_portfolio(tickers, weights, start_date, end_date)
    print(portfolio_metrics)
    # Calculate diversification metrics
    diversification_metrics = calculate_diversification_metrics(tickers, weights, start_date, end_date)

    # Print the results
    print("Covariance Matrix:")
    print(diversification_metrics['Covariance Matrix'])
    
    print("\nCorrelation Matrix:")
    print(diversification_metrics['Correlation Matrix'])
    
    print(f"\nPortfolio Variance: {diversification_metrics['Portfolio Variance']:.6f}")
    print(f"Portfolio Volatility (Annualized): {diversification_metrics['Portfolio Volatility']:.6f}")
    print(f"Diversification Ratio: {diversification_metrics['Diversification Ratio']:.6f}")


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed

{'Expected Return': 0.38879328625727705, 'Portfolio Volatility': 0.2789047227282473, 'Sharpe Ratio': 1.3581458304180702, 'Value at Risk (VaR)': -0.02729831622068925, 'Portfolio Beta': 1.3504096659293823}
Covariance Matrix:
Ticker      AMZN       NKE
Ticker                    
AMZN    0.077788  0.020361
NKE     0.020361  0.119709

Correlation Matrix:
Ticker      AMZN       NKE
Ticker                    
AMZN    1.000000  0.210995
NKE     0.210995  1.000000

Portfolio Variance: 0.077788
Portfolio Volatility (Annualized): 0.278905
Diversification Ratio: 1.000000
